In [ ]:
from simpletransformers.classification import MultiLabelClassificationArgs, MultiLabelClassificationModel
from simpletransformers.seq2seq import *
import sklearn
import pandas as pd
import logging


logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

In [ ]:
from util import *

In [1]:
tier_translate = {'AnalysisAndModeling': 'Analysis and Modeling',
                  'AnatomicalTarget': 'Anatomical Target',
                  'Imaging':'Imaging',
                'Manufacturing':'Manufacturing',
 'SpecificationofUse': 'Specification of Use',
 'SurgicalMethod': 'Surgical Method',
 'AnalysisAndModeling_3DModeling': 'Analysis and Modeling: 3D Modeling',
 'AnatomicalTarget_LowerExtremity': 'Anatomical Target: Lower Extremity',
 'AnatomicalTarget_Torso': 'Anatomical Target: Torso',
 'AnatomicalTarget_UpperExtremity': 'Anatomical Target: Upper Extremity',
 'Imaging_CT': 'Imaging: CT',
 'Imaging_MRI': 'Imaging: MRI',
 'Imaging_Ultrasound': 'Imaging: Ultrasound',
 'Manufacturing_AdditiveManufacturing': 'Manufacturing: Additive Manufacturing',
 'PersonalizedProduct_Guide/Jig': 'Personalized Product: Guide or Jig',
 'PersonalizedProduct_Implant': 'Personalized Product: Implant',
 'SpecificationofUse_Disease': 'Specification of Use: Disease',
 'SpecificationofUse_JointReplacement': 'Specification of Use: Joint Replacement',
 'AnatomicalTarget_LowerExtremity_Hip': 'Anatomical Target: Lower Extremity - Hip',
 'AnatomicalTarget_LowerExtremity_Knee': 'Anatomical Target: Lower Extremity - Knee'}

In [5]:
list(sorted(tier_translate.items()))

[('AnalysisAndModeling', 'Analysis and Modeling'),
 ('AnalysisAndModeling_3DModeling', 'Analysis and Modeling: 3D Modeling'),
 ('AnatomicalTarget', 'Anatomical Target'),
 ('AnatomicalTarget_LowerExtremity', 'Anatomical Target: Lower Extremity'),
 ('AnatomicalTarget_LowerExtremity_Hip',
  'Anatomical Target: Lower Extremity - Hip'),
 ('AnatomicalTarget_LowerExtremity_Knee',
  'Anatomical Target: Lower Extremity - Knee'),
 ('AnatomicalTarget_Torso', 'Anatomical Target: Torso'),
 ('AnatomicalTarget_UpperExtremity', 'Anatomical Target: Upper Extremity'),
 ('Imaging', 'Imaging'),
 ('Imaging_CT', 'Imaging: CT'),
 ('Imaging_MRI', 'Imaging: MRI'),
 ('Imaging_Ultrasound', 'Imaging: Ultrasound'),
 ('Manufacturing', 'Manufacturing'),
 ('Manufacturing_AdditiveManufacturing',
  'Manufacturing: Additive Manufacturing'),
 ('PersonalizedProduct_Guide/Jig', 'Personalized Product: Guide or Jig'),
 ('PersonalizedProduct_Implant', 'Personalized Product: Implant'),
 ('SpecificationofUse', 'Specification of

In [6]:
dict([('AnalysisAndModeling', 'Analysis and Modeling'),
 ('AnalysisAndModeling_3DModeling', 'Analysis and Modeling: 3D Modeling'),
 ('AnatomicalTarget', 'Anatomical Target'),
 ('AnatomicalTarget_LowerExtremity', 'Anatomical Target: Lower Extremity'),
 ('AnatomicalTarget_LowerExtremity_Hip',
  'Anatomical Target: Lower Extremity - Hip'),
 ('AnatomicalTarget_LowerExtremity_Knee',
  'Anatomical Target: Lower Extremity - Knee'),
 ('AnatomicalTarget_Torso', 'Anatomical Target: Torso'),
 ('AnatomicalTarget_UpperExtremity', 'Anatomical Target: Upper Extremity'),
 ('Imaging', 'Imaging'),
 ('Imaging_CT', 'Imaging: CT'),
 ('Imaging_MRI', 'Imaging: MRI'),
 ('Imaging_Ultrasound', 'Imaging: Ultrasound'),
 ('Manufacturing', 'Manufacturing'),
 ('Manufacturing_AdditiveManufacturing',
  'Manufacturing: Additive Manufacturing'),
 ('PersonalizedProduct_Guide/Jig', 'Personalized Product: Guide or Jig'),
 ('PersonalizedProduct_Implant', 'Personalized Product: Implant'),
 ('SpecificationofUse', 'Specification of Use'),
 ('SpecificationofUse_Disease', 'Specification of Use: Disease'),
 ('SpecificationofUse_JointReplacement',
  'Specification of Use: Joint Replacement'),
 ('SurgicalMethod', 'Surgical Method')])

{'AnalysisAndModeling': 'Analysis and Modeling',
 'AnalysisAndModeling_3DModeling': 'Analysis and Modeling: 3D Modeling',
 'AnatomicalTarget': 'Anatomical Target',
 'AnatomicalTarget_LowerExtremity': 'Anatomical Target: Lower Extremity',
 'AnatomicalTarget_LowerExtremity_Hip': 'Anatomical Target: Lower Extremity - Hip',
 'AnatomicalTarget_LowerExtremity_Knee': 'Anatomical Target: Lower Extremity - Knee',
 'AnatomicalTarget_Torso': 'Anatomical Target: Torso',
 'AnatomicalTarget_UpperExtremity': 'Anatomical Target: Upper Extremity',
 'Imaging': 'Imaging',
 'Imaging_CT': 'Imaging: CT',
 'Imaging_MRI': 'Imaging: MRI',
 'Imaging_Ultrasound': 'Imaging: Ultrasound',
 'Manufacturing': 'Manufacturing',
 'Manufacturing_AdditiveManufacturing': 'Manufacturing: Additive Manufacturing',
 'PersonalizedProduct_Guide/Jig': 'Personalized Product: Guide or Jig',
 'PersonalizedProduct_Implant': 'Personalized Product: Implant',
 'SpecificationofUse': 'Specification of Use',
 'SpecificationofUse_Disease': '

In [ ]:
def array_labels(row, tiers=all_tiers,):
    labels = []
    for c in tiers:
        if c in row and row[c]:
            if c in tier_translate:
                labels.append(tier_translate[c])
    return list(sorted(set(labels)))

In [ ]:
def array_labels_textual(row, tiers=all_tiers,):
    labels = []
    for c in tiers:
        if c in row and row[c]:
            if c in tier_translate:
                labels.append(tier_translate[c])
    return ", ".join(sorted(set(labels)))

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer

In [ ]:
training_set = pd.read_json("training_set.json.gz", lines=True, orient="records")
testing_set = pd.read_json("testing_set.json.gz", lines=True, orient="records")


training_set['target_text'] = training_set[all_tiers_100].apply(array_labels_textual,axis=1)


testing_set['target_text'] = testing_set[all_tiers_100].apply(array_labels_textual,axis=1)

In [ ]:
train_df = pd.DataFrame()
train_df["input_text"] = training_set.abstract
train_df["target_text"] = training_set.target_text

In [ ]:
test_df = pd.DataFrame()
test_df["input_text"] = testing_set.abstract
test_df['target_text'] = testing_set.target_text

In [ ]:
model_args = Seq2SeqArgs()
model_args.fp16 = True
model_args.use_early_stopping = True
model_args.early_stopping_delta = 0.01
#model_args.early_stopping_metric = "LRAP"
#model_args.early_stopping_metric_minimize = False
model_args.early_stopping_patience = 5
model_args.evaluate_during_training_steps = 100
model_args.num_train_epochs = 50
model_args.do_lower_case = False
model_args.dataloader_num_workers=0
model_args.use_multiprocessing=False
model_args.overwrite_output_dir = True
model_args.evaluate_during_training=True
model_args.evaluate_during_training_silent=False
model_args.evaluate_during_training_verbose=True
model_args.use_cached_eval_features=True
model_args.evaluate_each_epoch=True
#model_args.max_seq_length=None
model_args.train_batch_size=8
model_args.save_best_model=True
model_args.save_model_every_epoch=False
model_args.evaluate_generated_text = True
#sliding_windows=True

model = Seq2SeqModel(
    encoder_decoder_type="bart",
    encoder_decoder_name="facebook/bart-large",
    args=model_args)


In [ ]:
model.train_model(train_df, eval_data=test_df)

In [ ]:
model.

In [ ]:
from sklearn.metrics import *

In [ ]:
result = model.eval_model(test_df)

In [ ]:
result

In [ ]:
print(model.predict(testing_set.abstract[0]))

In [ ]:
import numpy as np
np.mean(model_outputs)

In [ ]:
print(classification_report(testing_labels, model_outputs > 0.2481697, target_names=mlb.classes_))